In [ ]:
import numpy as np
import emcee
import os
from scipy.interpolate import RegularGridInterpolator
from multiprocessing import Pool
import matplotlib.pyplot as plt
import corner

# ====================== 物理常数和初始化 ======================
HBARC = 197.327  # MeV·fm 转换常数
M_NUCLEON = 938.0  # MeV/c²

class SpinMatrixCalculator:
    """计算自旋态矩阵元 <χ_i|χ_j> (论文附录E)"""
    def __init__(self):
        self._init_static_matrix()
    
    def _init_static_matrix(self):
        """初始化非动量依赖的矩阵元"""
        self.matrix = np.zeros((8, 8), dtype=complex)
        
        # 对角元 (论文式43)
        diag = [1.0, 1.0, 1.5, 0.5, 0.5, 1.0, 1.0, 9/20]
        np.fill_diagonal(self.matrix, diag)
        
        # 非对角元 (论文附录E)
        self.matrix[0,5] = self.matrix[5,0] = -1/3                   # <1|6>
        self.matrix[0,6] = self.matrix[6,0] = -1/3                   # <1|7>
        self.matrix[2,3] = self.matrix[3,2] = 1/np.sqrt(2)           # <3|4>
        self.matrix[2,4] = self.matrix[4,2] = -0.5j                  # <3|5>
        self.matrix[3,4] = self.matrix[4,3] = 0.5*(1-1j)             # <4|5>
        
    def get_matrix_element(self, i, j, p_vec, q_vec):
        """获取考虑动量依赖的自旋矩阵元"""
        # 转换为0-based索引
        i_idx, j_idx = i-1, j-1
        
        # 基础值
        value = self.matrix[i_idx, j_idx]
        
        # 处理动量依赖的项
        if {i,j} == {3,3}:  # <χ3|χ3> = 1.5 * |(p×q)·σ|²/(p²q²)
            cross = np.linalg.norm(np.cross(p_vec, q_vec))**2
            value = 1.5 * cross / (np.linalg.norm(p_vec)**2 * np.linalg.norm(q_vec)**2)
            
        elif {i,j} == {4,4}:  # <χ4|χ4> = 0.5 * [|p×q|² + |σ×(p×q)|²]/(p²q²)
            cross = np.cross(p_vec, q_vec)
            value = 0.5 * (np.linalg.norm(cross)**2 + np.linalg.norm(np.cross(cross, cross))**2) \
                    / (np.linalg.norm(p_vec)**2 * np.linalg.norm(q_vec)**2)
        
        return value

# ====================== 波函数计算核心 ======================
class He3WaveFunction:
    def __init__(self, data_dir):
        # 加载网格和phi函数
        self._load_data(data_dir)
        
        # 初始化自旋矩阵计算器
        self.spin_matrix = SpinMatrixCalculator()
    
    def _load_data(self, data_dir):
        """加载网格数据和phi函数插值器"""
        # 读取网格文件
        def read_grid(filename):
            with open(os.path.join(data_dir, filename), 'r') as f:
                lines = [line.strip() for line in f if line.strip()]
            n = int(lines[0])
            return np.array([float(x.replace('D','E')) for x in lines[1:n+1]])
        
        self.q_grid = read_grid('qpx.dat')  # fm⁻¹
        self.p_grid = read_grid('qpx.dat')
        self.x_grid = read_grid('qpx.dat')  # cosθ
        
        # 读取phi函数 (1-8)
        self.phi_interps = {}
        for i in range(1,9):
            data = np.loadtxt(os.path.join(data_dir, f'he3.av18.phi{i}.dat'))
            data = data.reshape((len(self.x_grid), len(self.p_grid), len(self.q_grid))
            self.phi_interps[i] = RegularGridInterpolator(
                (self.q_grid, self.p_grid, self.x_grid),
                data.T,  # 转置为(q,p,x)顺序
                bounds_error=False,
                fill_value=0.0
            )
    
    def _jacobi_to_momentum(self, r, s, theta):
        """将Jacobi坐标转换为动量空间变量（简化模型）"""
        p = 1.0 / (r + 1e-10)  # 近似转换
        q = 1.0 / (s + 1e-10)
        return p, q, np.array([p*np.sin(theta), 0, p*np.cos(theta)]), \
               np.array([q*np.sin(theta), 0, q*np.cos(theta)])
    
    def calculate_density(self, r, s, theta):
        """计算包含自旋的完整概率密度 |Ψ|² * r²s²"""
        # 转换为动量空间
        p, q, p_vec, q_vec = self._jacobi_to_momentum(r, s, theta)
        x = np.cos(theta)
        
        # 计算所有phi_i(p,q,x)
        phi = np.array([interp((q, p, x)) for i, interp in sorted(self.phi_interps.items())])
        
        # 计算自旋矩阵
        spin_mat = np.zeros((8,8), dtype=complex)
        for i in range(1,9):
            for j in range(1,9):
                spin_mat[i-1,j-1] = self.spin_matrix.get_matrix_element(i, j, p_vec, q_vec)
        
        # 波函数模方
        psi_sq = np.einsum('i,ij,j->', phi.conj(), spin_mat, phi).real
        
        # 乘以相空间体积元
        return psi_sq * (r**2 * s**2 * np.sin(theta))

# ====================== MCMC采样器 ======================
class He3Sampler:
    def __init__(self, data_dir, n_walkers=32):
        self.wavefunction = He3WaveFunction(data_dir)
        self.n_walkers = n_walkers
    
    def _log_probability(self, params):
        r, s, theta = params
        if r <= 0 or s <= 0 or theta < 0 or theta > np.pi:
            return -np.inf
        return np.log(self.wavefunction.calculate_density(r, s, theta) + 1e-16)
    
    def run_sampling(self, n_steps=5000, burn_in=1000):
        # 初始walker位置
        initial = np.column_stack([
            np.random.uniform(0.5, 3.0, self.n_walkers),  # r (fm)
            np.random.uniform(0.5, 3.0, self.n_walkers),  # s (fm)
            np.random.uniform(0, np.pi, self.n_walkers)   # θ (rad)
        ])
        
        # 设置并行采样器
        with Pool() as pool:
            sampler = emcee.EnsembleSampler(
                self.n_walkers, 3, self._log_probability,
                pool=pool
            )
            sampler.run_mcmc(initial, n_steps, progress=True)
        
        # 处理样本
        samples = sampler.get_chain(discard=burn_in, flat=True)
        return samples, sampler

# ====================== 坐标转换与可视化 ======================
def jacobi_to_cartesian(samples):
    """将Jacobi坐标(r,s,θ)转换为三个核子的笛卡尔坐标"""
    cartesian = []
    for r, s, theta in samples:
        # 粒子1和2沿x轴对称放置
        x1, y1 = +r/2, 0
        x2, y2 = -r/2, 0
        
        # 粒子3在x-y平面
        x3 = s * np.cos(theta)
        y3 = s * np.sin(theta)
        
        # 移动到质心系
        x_cm = (x1 + x2 + x3)/3
        y_cm = (y1 + y2 + y3)/3
        
        cartesian.append([
            [x1-x_cm, y1-y_cm, 0],  # 核子1
            [x2-x_cm, y2-y_cm, 0],  # 核子2
            [x3-x_cm, y3-y_cm, 0]   # 核子3
        ])
    return np.array(cartesian)

def plot_results(samples):
    """绘制采样结果诊断图"""
    # 1. Jacobi坐标分布
    fig, axes = plt.subplots(1, 3, figsize=(15,4))
    axes[0].hist(samples[:,0], bins=50, label='r (fm)')
    axes[1].hist(samples[:,1], bins=50, label='s (fm)')
    axes[2].hist(samples[:,2], bins=50, label='θ (rad)')
    for ax in axes: ax.legend()
    
    # 2. 二维相关性
    plt.figure(figsize=(10,8))
    corner.corner(samples, labels=['r (fm)', 's (fm)', 'θ (rad)'], 
                  show_titles=True, quantiles=[0.16, 0.5, 0.84])

# ====================== 主程序 ======================
if __name__ == "__main__":
    # 配置参数
    DATA_DIR = "/path/to/your/data"  # 替换为实际路径
    N_WALKERS = 32
    N_STEPS = 3000
    BURN_IN = 800
    
    # 运行采样
    print("=== 开始He3核子位置采样 ===")
    sampler = He3Sampler(DATA_DIR, n_walkers=N_WALKERS)
    samples, emcee_sampler = sampler.run_sampling(n_steps=N_STEPS, burn_in=BURN_IN)
    
    # 分析结果
    print(f"\n采样完成！有效样本数: {len(samples)}")
    print(f"平均接受率: {np.mean(emcee_sampler.acceptance_fraction):.2f}")
    
    # 坐标转换
    cartesian_coords = jacobi_to_cartesian(samples)
    np.save("he3_samples.npy", cartesian_coords)
    print("笛卡尔坐标已保存到 he3_samples.npy")
    
    # 可视化
    plot_results(samples)
    plt.show()